In [1]:
!export PYTHONPATH=.:$PYTHONPATH

In [238]:
from odi.feature_engg import feature_extractor as fe
from odi.feature_engg import util as cricutil
from odi.preprocessing import rank
from odi.retrain import create_train_test_ci as ctt
from odi.model_util import odi_util as outil
import pandas as pd
from datetime import datetime
import math
import numpy as np
from tqdm.notebook import tqdm
import os

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,accuracy_score,precision_recall_fscore_support

In [3]:
import pickle

In [15]:
train_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.first_innings_base_train_x),'rb'))
train_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.first_innings_base_train_y),'rb'))

test_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.first_innings_base_test_x),'rb'))
test_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.first_innings_base_test_y),'rb'))

cols = pickle.load(open(os.path.join(outil.DEV_DIR, ctt.first_innings_base_columns), 'rb'))

In [9]:
train_x.shape

(460, 56)

In [10]:
test_x.shape

(58, 56)

In [16]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [18]:
# pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
# sfs = SequentialFeatureSelector(pipe, n_features_to_select=10)
# sfs.fit(train_x, train_y)

In [29]:
train_df = pd.DataFrame(train_x)
train_df.columns= cols
train_df['runs']=train_y

test_df = pd.DataFrame(test_x)
test_df.columns = cols
test_df['runs']=test_y

In [28]:
train_df.isnull().sum()

team_score                                        0
opponent_score                                    0
opponent_base                                     0
opponent_trend                                    0
opponent_trend_predict                            0
opponent_mean                                     0
location_base                                     0
location_trend                                    0
location_trend_predict                            0
location_mean                                     0
current_base                                      0
current_trend                                     0
current_trend_predict                             0
current_mean                                      0
run_factor                                        0
score_sum_7                                       0
score_sum_6                                       0
score_sum_5                                       0
score_sum_4                                       0
score_sum_3 

In [30]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [39]:
#np.array(train_df)

In [35]:
pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
sfs = SequentialFeatureSelector(pipe, n_features_to_select=10)
sfs.fit(train_df.drop(columns='runs'), train_df['runs'])

SequentialFeatureSelector(estimator=Pipeline(steps=[('scaler',
                                                     StandardScaler()),
                                                    ('regression',
                                                     LinearRegression())]),
                          n_features_to_select=10)

In [37]:
print("=======overall improtance=========")
selected_cols = []
for idx in np.where(sfs.get_support())[0]:
    print(cols[idx])
    selected_cols.append(cols[idx])

=======overall improtance=========
location_base
current_trend_predict
current_mean
score_sum_weighted_by_effectiveness_4
score_sum_weighted_by_log_4
score_mean_weighted_by_correlation_4
player_1
player_3
bowler_score_sum
bowler_score_mean


In [38]:
train_x_selected = np.array(train_df[selected_cols])
test_x_selected = np.array(test_df[selected_cols])

train_y_selected = np.array(train_df['runs'])
test_y_selected = np.array(test_df['runs'])

In [175]:
train_pipe = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])
#train_pipe = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('svc',SVR(C=0.5, epsilon=0.2))])



In [176]:
train_pipe.fit(train_x_selected,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [177]:
train_predict = train_pipe.predict(train_x_selected)
test_predict = train_pipe.predict(test_x_selected)

In [178]:
mean_absolute_error(train_y_selected,train_predict), mean_absolute_percentage_error(train_y_selected,train_predict)



(42.97436195664053, 0.1885279962294184)

In [179]:
mean_absolute_error(test_y_selected,test_predict), mean_absolute_percentage_error(test_y_selected,test_predict)


(51.22272288827973, 0.22635754649833756)

In [156]:
SS_Residual = sum((train_y_selected-train_predict)**2)    
SS_Total = sum((train_y_selected-np.mean(train_y_selected))**2) 
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(train_y_selected)-1)/(len(train_y_selected)-train_x_selected.shape[1]-1)

r_squared,adjusted_r_squared



(0.21101628779368198, 0.1928369395861631)

In [80]:
train_x_selected_all = np.array(train_df[cols])
test_x_selected_all = np.array(test_df[cols])

train_y_selected = np.array(train_df['runs'])
test_y_selected = np.array(test_df['runs'])

In [131]:
train_pipe_all = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])

In [132]:
train_pipe_all.fit(train_x_selected_all,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [133]:
train_predict_all = train_pipe_all.predict(train_x_selected_all)
test_predict_all = train_pipe_all.predict(test_x_selected_all)

In [134]:
mean_absolute_error(train_y_selected,train_predict_all), mean_absolute_percentage_error(train_y_selected,train_predict_all)



(42.1916260489623, 0.18332192433858666)

In [135]:
mean_absolute_error(test_y_selected,test_predict_all), mean_absolute_percentage_error(test_y_selected,test_predict_all)


(51.556133449736926, 0.22390662757650026)

In [215]:
cols

['team_score',
 'opponent_score',
 'opponent_base',
 'opponent_trend',
 'opponent_trend_predict',
 'opponent_mean',
 'location_base',
 'location_trend',
 'location_trend_predict',
 'location_mean',
 'current_base',
 'current_trend',
 'current_trend_predict',
 'current_mean',
 'run_factor',
 'score_sum_7',
 'score_sum_6',
 'score_sum_5',
 'score_sum_4',
 'score_sum_3',
 'score_max',
 'score_sum_weighted_by_correlation',
 'score_sum_weighted_by_contribution',
 'score_sum_weighted_by_effectiveness',
 'score_sum_weighted_by_log',
 'score_mean_weighted_by_correlation',
 'score_mean_weighted_by_contribution',
 'score_mean_weighted_by_effectiveness',
 'score_mean_weighted_by_log',
 'score_sum_weighted_by_correlation_4',
 'score_sum_weighted_by_contribution_4',
 'score_sum_weighted_by_effectiveness_4',
 'score_sum_weighted_by_log_4',
 'score_mean_weighted_by_correlation_4',
 'score_mean_weighted_by_contribution_4',
 'score_mean_weighted_by_effectiveness_4',
 'score_mean_weighted_by_log_4',
 's

In [223]:
# handpicked_cols = ['score_sum_weighted_by_correlation','score_sum_weighted_by_contribution',
#                    'team_score','bowler_score_sum',
#                   'location_base','current_trend_predict','current_mean']

handpicked_cols = ['team_score','opponent_score','score_sum_weighted_by_correlation','score_sum_weighted_by_contribution',
                   'score_sum_weighted_by_effectiveness','bowler_score_sum_weighted_by_contribution','bowler_score_sum_weighted_by_wr_contribution',
                   'team_score','bowler_score_sum',
                  'location_base','current_trend_predict','current_mean']



In [224]:
train_x_selected_hp = np.array(train_df[handpicked_cols])
test_x_selected_hp = np.array(test_df[handpicked_cols])

train_y_selected = np.array(train_df['runs'])
test_y_selected = np.array(test_df['runs'])

In [225]:
train_pipe_hp = Pipeline([('scaler', StandardScaler()), ('polynom',PolynomialFeatures(1)),('regression', LinearRegression())])

In [226]:
train_pipe_hp.fit(train_x_selected_hp,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [227]:
train_predict_hp = train_pipe_hp.predict(train_x_selected_hp)
test_predict_hp = train_pipe_hp.predict(test_x_selected_hp)

In [228]:
mean_absolute_error(train_y_selected,train_predict_hp), mean_absolute_percentage_error(train_y_selected,train_predict_hp)



(44.106140827044705, 0.19740297483154506)

In [229]:
mean_absolute_error(test_y_selected,test_predict_hp), mean_absolute_percentage_error(test_y_selected,test_predict_hp)



(53.42038480330737, 0.22505873059786413)

# Classification 


In [235]:
q1=np.quantile(train_y,0.25)
q2 = np.quantile(train_y,0.50)
q3 = np.quantile(train_y,0.75)
iqr = q3-q1
anomaly_h = q3 + 1.5*iqr
anomaly_l = q1 - 1.5*iqr

In [236]:
q1,q2,q3,anomaly_l,anomaly_h

(223.0, 267.5, 303.0, 103.0, 423.0)

In [240]:
def set_quantile(val,q1,q2,q3):
    iqr = q3-q1
    anomaly_h = q3 + 1.5*iqr
    anomaly_l = q1 - 1.5*iqr
    target_q = None
    if val > anomaly_h:
        target_q = 6
    elif val > q3:
        target_q = 5
    elif val > q2:
        target_q = 4
    elif val > q1:
        target_q = 3
    elif val > anomaly_l:
        target_q = 2
    else:
        target_q = 1
        
    return target_q

In [242]:
train_df['target_q']=train_df['runs'].apply(lambda x:set_quantile(x,q1,q2,q3))
test_df['target_q']=test_df['runs'].apply(lambda x:set_quantile(x,q1,q2,q3))

In [243]:
train_df.columns

Index(['team_score', 'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'run_factor', 'score_sum_7', 'score_sum_6',
       'score_sum_5', 'score_sum_4', 'score_sum_3', 'score_max',
       'score_sum_weighted_by_correlation',
       'score_sum_weighted_by_contribution',
       'score_sum_weighted_by_effectiveness', 'score_sum_weighted_by_log',
       'score_mean_weighted_by_correlation',
       'score_mean_weighted_by_contribution',
       'score_mean_weighted_by_effectiveness', 'score_mean_weighted_by_log',
       'score_sum_weighted_by_correlation_4',
       'score_sum_weighted_by_contribution_4',
       'score_sum_weighted_by_effectiveness_4', 'score_sum_weighted_by_log_4',
       'score_mean_weighted_by_correlation_4',
       'score_mean_weighted_by_contribution

In [260]:
train_x_bucket = np.array(train_df.drop(columns=['runs','target_q']))
test_x_bucket = np.array(test_df.drop(columns=['runs','target_q']))

train_y_bucket = np.array(train_df['target_q'])
test_y_bucket = np.array(test_df['target_q'])

In [261]:
train_pipe_bucket = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])



In [262]:
train_pipe_bucket.fit(train_x_bucket,train_y_bucket)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [263]:
train_predict_bucket = train_pipe_bucket.predict(train_x_bucket)
test_predict_bucket = train_pipe_bucket.predict(test_x_bucket)

In [254]:
mean_absolute_error(train_y_bucket,train_predict_bucket),mean_absolute_error(test_y_bucket,test_predict_bucket)



(8.76701956973663e-15, 3.267352139482053)

In [249]:
accuracy_score(train_y_bucket,np.round(train_predict_bucket)),accuracy_score(test_y_bucket,np.round(test_predict_bucket))



(0.3348314606741573, 0.3103448275862069)

In [250]:
precision_recall_fscore_support(train_y_bucket,np.round(train_predict_bucket),average='micro'),precision_recall_fscore_support(test_y_bucket,np.round(test_predict_bucket),average='micro')



((0.3348314606741573, 0.3348314606741573, 0.3348314606741573, None),
 (0.3103448275862069, 0.3103448275862069, 0.3103448275862069, None))

In [281]:
train_pipe_cls = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LogisticRegression())])




In [282]:
sfs = SequentialFeatureSelector(train_pipe_cls, n_features_to_select=10)
sfs.fit(train_x_bucket, train_y_bucket)

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/anaconda3/envs/tensor230/lib/python3.6/

...

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

SequentialFeatureSelector(estimator=Pipeline(steps=[('scaler',
                                                     StandardScaler()),
                                                    ('polynom',
                                                     PolynomialFeatures(degree=1)),
                                                    ('regression',
                                                     LogisticRegression())]),
                          n_features_to_select=10)

In [283]:
print("=======overall improtance=========")
selected_cls_cols = []
for idx in np.where(sfs.get_support())[0]:
    print(cols[idx])
    selected_cls_cols.append(cols[idx])

=======overall improtance=========
opponent_score
current_trend_predict
score_sum_7
score_sum_3
score_sum_weighted_by_log
score_mean_weighted_by_log
score_sum_weighted_by_effectiveness_4
score_mean_7
score_mean_3
player_1


In [284]:
np.where(sfs.get_support())[0]

array([ 1, 12, 15, 19, 24, 28, 31, 37, 41, 42])

In [285]:
train_pipe_cls.fit(train_x_bucket[:,np.where(sfs.get_support())[0]],train_y_bucket)



/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LogisticRegression())])

In [286]:
train_predict_cls = train_pipe_cls.predict(train_x_bucket[:,np.where(sfs.get_support())[0]])
test_predict_cls = train_pipe_cls.predict(test_x_bucket[:,np.where(sfs.get_support())[0]])



In [287]:
accuracy_score(train_y_bucket,np.round(train_predict_cls)),accuracy_score(test_y_bucket,np.round(test_predict_cls))




(0.3887640449438202, 0.4482758620689655)

In [291]:
tq1 = np.quantile(train_df['team_score'],0.25)
tq2 = np.quantile(train_df['team_score'],0.50)
tq3 = np.quantile(train_df['team_score'],0.75)

In [292]:
train_df_q1 = train_df[train_df['team_score']<tq1]
train_df_q2 = train_df[train_df['team_score']<tq2]
train_df_q3 = train_df[train_df['team_score']<tq3]
train_df_q4 = train_df[train_df['team_score']>=tq3]

test_df_q1 = test_df[test_df['team_score']<tq1]
test_df_q2 = test_df[test_df['team_score']<tq2]
test_df_q3 = test_df[test_df['team_score']<tq3]
test_df_q4 = test_df[test_df['team_score']>=tq3]

In [294]:
test_df_q1.shape,test_df_q2.shape,test_df_q3.shape,test_df_q4.shape

((22, 58), (39, 58), (45, 58), (13, 58))

In [295]:
train_pipe_q1 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])
train_pipe_q2 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])
train_pipe_q3 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])
train_pipe_q4 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LinearRegression())])



In [296]:
train_x_q1 = np.array(train_df_q1[cols])
test_x_q1 = np.array(train_df_q1[cols])
train_y_q1 = np.array(train_df_q1['runs'])
test_y_q1 = np.array(train_df_q1['runs'])

train_x_q2 = np.array(train_df_q2[cols])
test_x_q2 = np.array(train_df_q2[cols])
train_y_q2 = np.array(train_df_q2['runs'])
test_y_q2 = np.array(train_df_q2['runs'])

train_x_q3 = np.array(train_df_q3[cols])
test_x_q3 = np.array(train_df_q3[cols])
train_y_q3 = np.array(train_df_q3['runs'])
test_y_q3 = np.array(train_df_q3['runs'])

train_x_q4 = np.array(train_df_q4[cols])
test_x_q4 = np.array(train_df_q4[cols])
train_y_q4 = np.array(train_df_q4['runs'])
test_y_q4 = np.array(train_df_q4['runs'])

In [297]:
train_pipe_q1.fit(train_x_q1,train_y_q1)
train_pipe_q2.fit(train_x_q2,train_y_q2)
train_pipe_q3.fit(train_x_q3,train_y_q3)
train_pipe_q4.fit(train_x_q4,train_y_q4)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [301]:
train_predict_q1 = train_pipe_q1.predict(train_x_q1)
train_predict_q2 = train_pipe_q2.predict(train_x_q2)
train_predict_q3 = train_pipe_q3.predict(train_x_q3)
train_predict_q4 = train_pipe_q4.predict(train_x_q4)

test_predict_q1 = train_pipe_q1.predict(test_x_q1)
test_predict_q2 = train_pipe_q2.predict(test_x_q2)
test_predict_q3 = train_pipe_q3.predict(test_x_q3)
test_predict_q4 = train_pipe_q4.predict(test_x_q4)

In [303]:
mean_absolute_percentage_error(train_y_q1,train_predict_q1), mean_absolute_percentage_error(test_y_q1,test_predict_q1)



(0.43111704338244894, 0.43111704338244894)

In [304]:
mean_absolute_percentage_error(train_y_q2,train_predict_q2), mean_absolute_percentage_error(test_y_q2,test_predict_q2)




(0.16704081616488015, 0.16704081616488015)

In [305]:
mean_absolute_percentage_error(train_y_q3,train_predict_q3), mean_absolute_percentage_error(test_y_q3,test_predict_q3)





(0.17343566821809026, 0.17343566821809026)

In [306]:
mean_absolute_percentage_error(train_y_q4,train_predict_q4), mean_absolute_percentage_error(test_y_q4,test_predict_q4)



(0.1539535408887592, 0.1539535408887592)